In [3]:
import pandas as pd 
DATASET = "amazon_reviews_us_Grocery_v1_00.tsv"
import logging
import warnings
import sys
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
logging.disable(sys.maxsize)



/usr/local/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
dataset = pd.read_table(DATASET, error_bad_lines=False, header=0, warn_bad_lines=False) 
# there are some malformed entries the dataset... let's ignore those for now


In [5]:
### Let's see what the dataset looks like... 
dataset.head()


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,42521656,R26MV8D0KG6QI6,B000SAQCWC,159713740,"The Cravings Place Chocolate Chunk Cookie Mix,...",Grocery,5,0.0,0.0,N,Y,Using these for years - love them.,"As a family allergic to wheat, dairy, eggs, nu...",2015-08-31
1,US,12049833,R1OF8GP57AQ1A0,B00509LVIQ,138680402,"Mauna Loa Macadamias, 11 Ounce Packages",Grocery,5,0.0,0.0,N,Y,Wonderful,"My favorite nut. Creamy, crunchy, salty, and ...",2015-08-31
2,US,107642,R3VDC1QB6MC4ZZ,B00KHXESLC,252021703,Organic Matcha Green Tea Powder - 100% Pure Ma...,Grocery,5,0.0,0.0,N,N,Five Stars,This green tea tastes so good! My girlfriend l...,2015-08-31
3,US,6042304,R12FA3DCF8F9ER,B000F8JIIC,752728342,15oz Raspberry Lyons Designer Dessert Syrup Sauce,Grocery,5,0.0,0.0,N,Y,Five Stars,I love Melissa's brand but this is a great sec...,2015-08-31
4,US,18123821,RTWHVNV6X4CNJ,B004ZWR9RQ,552138758,"Stride Spark Kinetic Fruit Sugar Free Gum, 14-...",Grocery,5,0.0,0.0,N,Y,Five Stars,good,2015-08-31


In [6]:
#lots of nan values... :( 
dataset.isnull().sum()

marketplace           0
customer_id           0
review_id             0
product_id            0
product_parent        0
product_title         0
product_category      0
star_rating          22
helpful_votes        23
total_votes          23
vine                 23
verified_purchase    23
review_headline      34
review_body          88
review_date          65
dtype: int64

In [7]:
# Lots of spammy duplicate reviews...might be worth checking if the same person is duplicating the reviews across multiple products

# Additions, because maybe same review_body is not duplicate i.e. "Good" as review.
x = dataset[dataset['review_body'].duplicated() == True]
x = x[dataset['customer_id'].duplicated() == True] # ADDED THIS
x = x[dataset['product_id'].duplicated() == True] # ADDED THIS
print(len(x))
x.head()

75144


,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
2323,US,32433311,R3DPQJFH05T0HS,B00OZYNGUS,228095196,Viva Naturals - The FINEST Raw Organic Chia Se...,Grocery,5,0.0,0.0,N,Y,Great product.,I've been using this to help keep me from snac...,2015-08-31
2330,US,130585,R3IOACXWPYTRUP,B00HZ7HSTW,885171844,Beantown Roasters K Cups Variety Packs,Grocery,5,0.0,1.0,N,Y,The Office like it!,These were purchased for an office and they lo...,2015-08-31
2401,US,14100648,RA8F0DF5OJH3J,B000WV0RW8,653213046,Healthworks Chia Seeds 3lb 6lb Parent,Grocery,5,0.0,0.0,N,Y,GREAT! Thanks So much and God bless,GREAT! Thanks So much and God bless! &#60;&#...,2015-08-31
2436,US,43554935,RK0TC9FRNUL55,B00HQ3ZPJA,220133938,Chia Seeds,Grocery,5,0.0,0.0,N,Y,Five Stars,As advertised.,2015-08-31
2541,US,168472,R2WFNGP0E0ZLXW,B006ZMYLKC,322601427,Keurig,Grocery,5,0.0,0.0,N,Y,Five Stars,Satisfied,2015-08-31


In [8]:
#remove NaNs from the dataset  for now...
dataset = dataset.dropna()

In [9]:
def filter_heuristic(row):
    '''
    determine if a row is a negative review based on its score. Potentially extend it to include sentiment analysis 
    and presence of specific negative words 
    '''
    return row["star_rating"].astype(int) <=3

In [10]:
negative_rows = dataset.loc[filter_heuristic]
negative_rows.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
5,US,23649464,RIG9AWFOGRDVO,B00AL6QBZ6,681475449,Herr's Popcorn Hot Cheese 1 Oz (Pack of 30),Grocery,2,1.0,1.0,N,Y,Not Happy,The popcorn was stale.,2015-08-31
9,US,19624355,R1ODXB3C9UP3NL,B00J074W94,2499702,"Orgain Organic Plant Based Protein Powder, Pac...",Grocery,1,1.0,3.0,N,N,Disgusting now and difficult on digestion,Used to be a decent product. Disgusting now a...,2015-08-31
17,US,22765168,R3T6TTD2IN0EFZ,B00XDXMLL2,971154239,"Skippy Creamy Peanut Butter, with Salted Caram...",Grocery,1,4.0,4.0,N,N,"1 Out Of 5 Of My Co-Workers Thought It Was ""Okay""",I bought this from a local super market on a w...,2015-08-31
23,US,35636887,R9MISLBRG08FX,B00DBSFXUA,294404974,"Keebler Town House Pita Crackers, 9.5 Ounce",Grocery,1,0.0,0.0,N,Y,pita crackers,not craze about these. nothing really wrong wi...,2015-08-31
26,US,12650237,R2A9O8CWZ1PP74,B0083GJKR2,868929824,"Eclipse Sugar Free Gum, Spearmint, 120 Piece B...",Grocery,3,0.0,0.0,N,Y,Three Stars,it's gum..,2015-08-31


In [11]:
sorted_negative_rows = negative_rows.sort_values("product_id")
sorted_negative_rows.head() # ADDITION JUST TO CHECK

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
1975011,US,14434517,R25ATZCV5FBJMV,0805470867,518591127,Communion Bread Pack of 500,Grocery,2,3.0,4.0,N,Y,The only thing my church has ever complained a...,I'm the pastor of a church plant and the only ...,2012-10-16
1365978,US,47145,RUI9CRY6RZNRB,0805470867,518591127,Communion Bread Pack of 500,Grocery,1.0,0.0,0.0,N,Y,I HATE IT,what is that thing<br />it doesn't crunch like...,2014-02-21
1961839,US,37403535,R37YYRMOE7K2TB,0805470867,518591127,Communion Bread Pack of 500,Grocery,1,1.0,1.0,N,Y,Communion bread,"I purchased 3 boxes, two of them the experatio...",2012-11-11
78777,US,41394716,R2QAW3WQRYVQF5,0805470867,518591127,Communion Bread Pack of 500,Grocery,3,0.0,0.0,N,Y,Not what expected,"This came in a flimsy cardboard box, squished....",2015-08-04
1845770,US,11312642,R3M9ORU1I9L2C3,0805470867,518591127,Communion Bread Pack of 500,Grocery,2,0.0,0.0,N,Y,Gross,These things taste nothing like the communion ...,2013-02-18


In [12]:
# def generate_wordcloud(text):
#     wordcloud = WordCloud(width = 5000, height = 5000, random_state=1, background_color='salmon', colormap='Pastel1',
#                           collocations=False, stopwords = STOPWORDS).generate(text)
#     plt.imshow(wordcloud)

In [13]:
# generate a naive word cloud without any preprocessing....
# generate_wordcloud(" ".join(sorted_negative_rows["review_body"]))

In [14]:
dataset.columns

Index(['marketplace', 'customer_id', 'review_id', 'product_id',
       'product_parent', 'product_title', 'product_category', 'star_rating',
       'helpful_votes', 'total_votes', 'vine', 'verified_purchase',
       'review_headline', 'review_body', 'review_date'],
      dtype='object')

In [15]:
reviews = dataset["review_headline"]
print(reviews.head())
print(sorted_negative_rows["review_headline"])

0    Using these for years - love them.
1                             Wonderful
2                            Five Stars
3                            Five Stars
4                            Five Stars
Name: review_headline, dtype: object
1975011    The only thing my church has ever complained a...
1365978                                            I HATE IT
1961839                                      Communion bread
78777                                      Not what expected
1845770                                                Gross
                                 ...                        
30387                          i hv to say I LOVE THIS CHIPS
1269                                           not very good
18038      Gross Chemical Taste. Nothing like Pumpkin Pie...
22590                                               One Star
2307691                                       Pretty Rough!!
Name: review_headline, Length: 445115, dtype: object


In [16]:
# if not nltk.find('corpora/wordnet'): # may need to download it if it's not in your pc
#     nltk.download('wordnet')



import pip
pip.main(['install', 'nltk'])
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
print(lemmatizer.lemmatize('went')) # Default is noun
print(lemmatizer.lemmatize('went', 'v')) # If "v", it works with verbs.# If needed later, I will spend some time to find the part of speech of each word, and use lemmatizer accordingly

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/navyazaveri/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


went
go


In [17]:
import pip 
pip.main(["install", "gensim"])
from gensim.parsing.preprocessing import STOPWORDS
import gensim

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [18]:
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS:
            result.append(WordNetLemmatizer().lemmatize(token, 'v')) # for now, lemmatizing only verbs
    return result

In [19]:
for review in reviews[:5]:
    print("Removed stopwords and lemmatized words")
    print(review + " becomes -->", preprocess(review))
# Here, I notice that the amount of starts is getting removed. I don't think it's bad because a review being "1 star"
# does not necessarily mean the product is faulty. I can either further remove all the word "stars" from the texts, 
# Or try to figure out how to keep numbers, if it's possible, on lemmatizing

Removed stopwords and lemmatized words
Using these for years - love them. becomes --> ['years', 'love']
Removed stopwords and lemmatized words
Wonderful becomes --> ['wonderful']
Removed stopwords and lemmatized words
Five Stars becomes --> ['star']
Removed stopwords and lemmatized words
Five Stars becomes --> ['star']
Removed stopwords and lemmatized words
Five Stars becomes --> ['star']


In [20]:
### I will run this on the first 1000 reviews because for now, because it will take long for more
less_reviews = reviews[:1000]
processed = []
for review in less_reviews:
    processed.append(preprocess(review))
print(processed) # All the words after applying lemmatization on the first 1000 reviews

[['years', 'love'], ['wonderful'], ['star'], ['star'], ['star'], ['happy'], ['star'], ['star'], ['great', 'taste'], ['disgust', 'difficult', 'digestion'], ['like', 'soy', 'sauce', 'll', 'like'], ['star'], ['star'], ['star'], ['star'], ['excellent'], ['yummy'], ['workers', 'think', 'okay'], ['delicious', 'tea', 'easy'], ['best', 'coconut', 'oil'], ['star'], ['star'], ['good', 'variety'], ['pita', 'crackers'], ['star'], ['nice', 'big', 'bottle'], ['star'], ['star'], ['star'], ['star'], ['love'], ['star'], ['fresh', 'great', 'tomato', 'base', 'meal'], ['star'], ['goodie', 'downunder'], [], ['far', 'like'], ['star'], ['instructions', 'sure'], ['lifesavers', 'eons', 'trust', 'quality', 'good', 'taste', 'mint'], ['recommend'], ['star'], ['star'], ['star'], ['wonderful', 'watermelon'], ['great', 'product'], ['good', 'flavor'], ['tasteless'], ['tea', 'great', 'enjoy', 'flavor'], ['favorite', 'tea'], ['good'], ['taste', 'great'], ['organic', 'brand', 'better', 'taste'], ['great', 'import', 'are

In [21]:
dictionary = gensim.corpora.Dictionary(processed) # construct word<->id mappings - it does it in alphabetical order
print(dictionary) # In the first 1000 reviews, we have 787 unique words after applying lemmatization in every word

Dictionary(787 unique tokens: ['love', 'years', 'wonderful', 'star', 'happy']...)


In [22]:
bow_corpus = [dictionary.doc2bow(review) for review in processed]
# Each entry in bow_corpus lists how many times each word id exists. 

print(bow_corpus) # bow_corpus shows the id mapping for each word, and the total occurences in the dict of that word
# in that document

[[(0, 1), (1, 1)], [(2, 1)], [(3, 1)], [(3, 1)], [(3, 1)], [(4, 1)], [(3, 1)], [(3, 1)], [(5, 1), (6, 1)], [(7, 1), (8, 1), (9, 1)], [(10, 2), (11, 1), (12, 1), (13, 1)], [(3, 1)], [(3, 1)], [(3, 1)], [(3, 1)], [(14, 1)], [(15, 1)], [(16, 1), (17, 1), (18, 1)], [(19, 1), (20, 1), (21, 1)], [(22, 1), (23, 1), (24, 1)], [(3, 1)], [(3, 1)], [(25, 1), (26, 1)], [(27, 1), (28, 1)], [(3, 1)], [(29, 1), (30, 1), (31, 1)], [(3, 1)], [(3, 1)], [(3, 1)], [(3, 1)], [(0, 1)], [(3, 1)], [(5, 1), (32, 1), (33, 1), (34, 1), (35, 1)], [(3, 1)], [(36, 1), (37, 1)], [], [(10, 1), (38, 1)], [(3, 1)], [(39, 1), (40, 1)], [(6, 1), (25, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1)], [(46, 1)], [(3, 1)], [(3, 1)], [(3, 1)], [(2, 1), (47, 1)], [(5, 1), (48, 1)], [(25, 1), (49, 1)], [(50, 1)], [(5, 1), (21, 1), (49, 1), (51, 1)], [(21, 1), (52, 1)], [(25, 1)], [(5, 1), (6, 1)], [(6, 1), (53, 1), (54, 1), (55, 1)], [(5, 1), (56, 1), (57, 1)], [(3, 1)], [(3, 1)], [(3, 1)], [(3, 1)], [(3, 1)], [(3, 1)], [(19, 

In [23]:
# By changing review_num, we can view the amount of appearances of that word in that specific review
review_num = 0
bow_doc_x = bow_corpus[review_num]

for i in range(len(bow_doc_x)):
    print("Word {} (\"{}\") appears {} time(s).".format(bow_doc_x[i][0], 
                                                     dictionary[bow_doc_x[i][0]], 
                                                     bow_doc_x[i][1]))

Word 0 ("love") appears 1 time(s).
Word 1 ("years") appears 1 time(s).


In [24]:
# Here, we give the bag of words we have (for now I have used first 1000 reviews), how many topics we want to get,
# the dictionary of the id <-> word mappings and how many passes(epochs) to perform, and it creates the model.
lda_model =  gensim.models.LdaModel(bow_corpus, 
                                   num_topics=20, 
                                   id2word=dictionary,                                    
                                   passes=10,
                                   ) 

In [25]:
for idx, topic in lda_model.print_topics(-1): # The words occuring in each class, and the weight given for that
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")

# I have noticed that every time I run this, the results below are different. This can either be good or bad.
# Some times when I run it, there is a class with the most weighted word being "dissapoint". If we want results that
# are the same every time, we can use a seed for reproducibility. Let me know! 

Topic: 0 
Words: 0.151*"product" + 0.110*"great" + 0.059*"good" + 0.046*"stuff" + 0.020*"expect" + 0.017*"nice" + 0.017*"brand" + 0.016*"expensive" + 0.013*"love" + 0.012*"stale"


Topic: 1 
Words: 0.075*"buy" + 0.031*"try" + 0.030*"perfect" + 0.024*"bad" + 0.019*"quality" + 0.019*"alternative" + 0.019*"sugar" + 0.018*"enjoy" + 0.015*"tea" + 0.013*"drink"


Topic: 2 
Words: 0.034*"healthy" + 0.028*"great" + 0.028*"tasty" + 0.026*"delicious" + 0.019*"local" + 0.019*"sugar" + 0.019*"hot" + 0.019*"meh" + 0.019*"save" + 0.019*"simply"


Topic: 3 
Words: 0.082*"yummy" + 0.042*"taste" + 0.027*"great" + 0.023*"better" + 0.023*"smell" + 0.022*"gum" + 0.016*"zevia" + 0.008*"thank" + 0.008*"flavor" + 0.008*"recommend"


Topic: 4 
Words: 0.070*"best" + 0.039*"nice" + 0.027*"product" + 0.024*"bar" + 0.023*"salt" + 0.016*"buy" + 0.016*"isn" + 0.016*"kind" + 0.016*"new" + 0.015*"ship"


Topic: 5 
Words: 0.153*"great" + 0.069*"flavor" + 0.049*"like" + 0.041*"taste" + 0.026*"wonderful" + 0.025*"best" 

In [26]:
# Here, and on the next cell, I run this on a review that was not used for training to find out how we can get results
# on unseen data
unseen_document = reviews[1001]
print(unseen_document)
preprocess(unseen_document)

Kirkland Coffee is a terrific daily brand coffee.


['kirkland', 'coffee', 'terrific', 'daily', 'brand', 'coffee']

In [27]:
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

In [28]:
lda_model[bow_vector] # How likely each class is. Because the review is small, some classes have very low probabilities.
# I think that is good! Because there is a high probability to exist in the rest. But again, the data I have trained
# is coarse for now, so no guaranteed results (yet).

[(0, 0.010000201),
 (1, 0.0100002),
 (2, 0.0100002),
 (3, 0.0100002),
 (4, 0.0100002),
 (5, 0.0100002),
 (6, 0.0100002),
 (7, 0.40999156),
 (8, 0.0100002),
 (9, 0.0100002),
 (10, 0.0100002),
 (11, 0.0100002),
 (12, 0.0100002),
 (13, 0.0100002),
 (14, 0.010000201),
 (15, 0.0100002),
 (16, 0.0100002),
 (17, 0.0100002),
 (18, 0.41000482),
 (19, 0.0100002)]

In [29]:
# Sorts the most probable classes and gives some details on weights, words and probabilities
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Index: {}\nScore: {}\t Topic: {}".format(index, score, lda_model.print_topic(index, 5)))

Index: 18
Score: 0.4100048243999481	 Topic: 0.170*"taste" + 0.089*"great" + 0.043*"good" + 0.042*"coffee" + 0.042*"like"
Index: 7
Score: 0.40999162197113037	 Topic: 0.040*"organic" + 0.036*"fresh" + 0.018*"terrific" + 0.018*"rice" + 0.018*"cake"
Index: 0
Score: 0.010000200942158699	 Topic: 0.151*"product" + 0.110*"great" + 0.059*"good" + 0.046*"stuff" + 0.020*"expect"
Index: 14
Score: 0.010000200942158699	 Topic: 0.038*"work" + 0.033*"great" + 0.016*"brand" + 0.016*"different" + 0.016*"win"
Index: 1
Score: 0.010000200010836124	 Topic: 0.075*"buy" + 0.031*"try" + 0.030*"perfect" + 0.024*"bad" + 0.019*"quality"
Index: 2
Score: 0.010000200010836124	 Topic: 0.034*"healthy" + 0.028*"great" + 0.028*"tasty" + 0.026*"delicious" + 0.019*"local"
Index: 3
Score: 0.010000200010836124	 Topic: 0.082*"yummy" + 0.042*"taste" + 0.027*"great" + 0.023*"better" + 0.023*"smell"
Index: 4
Score: 0.010000200010836124	 Topic: 0.070*"best" + 0.039*"nice" + 0.027*"product" + 0.024*"bar" + 0.023*"salt"
Index: 5
S

In [30]:
import pyLDAvis
import pyLDAvis
import pyLDAvis.gensim

# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, bow_corpus, dictionary)

LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10     0.446927 -0.058589       1        1  10.069118
18    -0.068008 -0.191304       2        1   7.858623
8     -0.016869  0.115102       3        1   6.304826
5     -0.068201 -0.148412       4        1   6.271582
0     -0.046095 -0.097243       5        1   6.019592
17    -0.013988  0.053086       6        1   5.675881
1     -0.024131  0.122607       7        1   5.565331
12    -0.033993  0.003461       8        1   4.459372
3     -0.002625 -0.031540       9        1   4.425724
4     -0.033057  0.086736      10        1   4.372656
14    -0.020932 -0.009801      11        1   4.369395
9     -0.034055  0.074666      12        1   4.360461
15    -0.028873 -0.047184      13        1   4.283728
16    -0.050934 -0.079723      14        1   4.195563
6     -0.053916 -0.046478      15        1   4.020935
11     0.135619  0.040673      16        1   3.827270
7     -0.020827  0.057162      17        1   3.683636
2     -0.033001  0.013282      18        1   3.631276
19    -0.016376  0.098460      19        1   3.554331
13    -0.016665  0.045038      20        1   3.050701, topic_info=          Term        Freq       Total Category  logprob  loglift
3         star  206.000000  206.000000  Default  30.0000  30.0000
0         love   42.000000   42.000000  Default  29.0000  29.0000
25        good   55.000000   55.000000  Default  28.0000  28.0000
6        taste   44.000000   44.000000  Default  27.0000  27.0000
5        great   90.000000   90.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
53      better    0.798514    9.836815  Topic20  -4.4918   0.9787
253       work    0.798514    7.902893  Topic20  -4.4918   1.1976
22        best    0.798514   27.343985  Topic20  -4.4918  -0.0437
14   excellent    0.798513   10.812212  Topic20  -4.4918   0.8841
52    favorite    0.798513    5.319466  Topic20  -4.4918   1.5934

[941 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
164      15  0.654381    absolute
452      15  0.654385  absolutely
629      17  0.651600        ache
94       17  0.651602    actually
291       7  0.850983         add
...     ...       ...         ...
601      15  0.654392   yumminess
15        9  0.885785       yummy
15       17  0.098421       yummy
323       9  0.856047       zevia
679       9  0.644386       zingy

[1107 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 19, 9, 6, 1, 18, 2, 13, 4, 5, 15, 10, 16, 17, 7, 12, 8, 3, 20, 14])